In [1]:
from FEnicS_base import *
import dolfinx
from dolfinx import mesh, fem, io, nls
from dolfinx.fem import VectorFunctionSpace
from mpi4py import MPI
import numpy as np
import ufl
import matplotlib.pyplot as plt

import jsonpickle as jp
import shutil
import re
import inspect
from tqdm import tqdm

def function_plots(
    functions,
    fig=None,
    ax=None,
    show_points=False,
):
    """Create plot from fem.Function
    
    Args:
        fig (plt.Figure): Figure
        lists (fem.Function, str): List of (u, title)
        points_on (bool): If true create scatter
    """
    if (fig or ax) is None:
        fig, ax = plt.subplots(facecolor='White')
        fig.set_size_inches(16, 8)
    for func in functions:
        x = func.function_space.tabulate_dof_coordinates()[:, 0]
        y = func.x.array
        cord = np.array([x, y])
        cord = cord[:, np.argsort(cord[0])]
        ax.plot(cord[0], cord[1], label=func.name, linewidth=1)
        if show_points: ax.scatter(cord[0], cord[1], s=0.5)
    ax.legend(
        bbox_to_anchor=(1.01, 0.5),
        borderaxespad=0,
        loc='center left',
    )
    return ax


In [3]:
# yapf: disable
U,G= 'U','G'
DOMAIN = mesh.create_interval(nx=100, comm=MPI.COMM_WORLD, points=[0, 1])
element = {
    U:ufl.FiniteElement(
        family='CG',
        cell=DOMAIN.ufl_cell(),
        degree=2),
    G:ufl.VectorElement(
        family='CG',
        cell=DOMAIN.ufl_cell(),
        degree=2)
    }
SPACE = FunctionSpace(
    mesh=DOMAIN,
    element=ufl.MixedElement(element[U], element[G]),
)
SUBSPACE = {
    U: SPACE.sub(0).collapse()[0],
    G: SPACE.sub(1).collapse()[0],
}
x = SpatialCoordinate(SPACE)[0]

FUNC, FUNC0 = Function(SPACE), Function(SPACE)
SUB_FUNC = {
    U: FUNC.sub(0),
    G: FUNC.sub(1),
}
INDIC = {
    U: split(FUNC)[0],
    G: split(FUNC)[1],
}
INDIC_0 = {
    U: split(FUNC0)[0],
    G: split(FUNC0)[1],
}

TIME = Constant(SUBSPACE[U], 0)

class Distribution_collection:

    def __init__(self, x: SpatialCoordinate, x0, slope):
        self.x = x
        self.x0 = x0
        self.slope = slope

    def create(self, kind: str):
        assert kind in Distribution_collection.get_kinds(False), 'Not implemented method'
        return getattr(self, kind)()

    @staticmethod
    def _kind_ready(func=None, get=False, l=set()):
        if not get:
            l.add(func.__name__)
            return func
        else:
            return l

    @classmethod
    def get_kinds(cls, view=True):
        kinds = cls._kind_ready(get=True)
        if not view:
            return kinds
        else:
            print('\n'.join(kinds))

    def _singP(self):
        return (1 + ufl.sign(self.slope)) / 2

    def _singM(self):
        return (1 - ufl.sign(self.slope)) / 2

    @_kind_ready
    def step(self):
        return conditional(self.x0 <= self.x, self._singP(), self._singM())

    @_kind_ready
    def sigmoid(self):
        a = self.slope * 5
        return 1 / (1 + exp(-a * (self.x - self.x0)))

    @_kind_ready
    def trapsharp(self):
        a = self.slope
        res = conditional(
            ufl.And(
                self.x0 - 1 / (abs(2 * a)) <= self.x,
                self.x < self.x0 + 1 / (abs(2 * a))
            ),
            a * (self.x - self.x0) + 0.5,
            0,
        )
        res += conditional(
            self.x0 + 1 / (2*a) <= self.x,
            self._singP(),
            self._singM(),
        )
        return res

    @_kind_ready
    def parab(self):
        a = self.slope * 5
        res = conditional(
            ufl.And(
                self.x0 - 1 / sqrt(abs(2 * a)) <= self.x,
                self.x < self.x0,
            ),
            a * (self.x - self.x0 + 1 / sqrt(abs(2 * a)))**2 + self._singM(),
            0
        )
        res += conditional(
            ufl.And(
                self.x0 <= self.x,
                self.x < self.x0 + 1 / sqrt(abs(2 * a)),
            ),
            -a * (self.x - self.x0 - 1 / sqrt(abs(2 * a)))**2 + self._singP(),
            0
        )
        res += conditional(
            self.x0 + 1 / (ufl.sign(a) * sqrt(abs(2 * a))) <= self.x,
            self._singP(),
            self._singM()
        )
        return res
# Distribution_collection(x=x, x0=0.2, slope=100).create(kind='sigmoid')

CONST = {
    'd': 0.01,
}

SURFACE = {
    U: 0,
    G: 0,
}
BCS = []


def create_equation():

    def create_facets():
        set_connectivity(DOMAIN)
        ds = Measure("ds", domain=DOMAIN)
        return ds

    dx = Measure('cell', subdomain_id='everywhere')
    dt = 0.01
    ds = create_facets()

    f, h = TestFunctions(SPACE)
    u, g = INDIC[U], INDIC[G]
    u0, g0 = INDIC_0[U], INDIC_0[G]

    eq_h = (1/dt) * (u-u0) * f * dx
    eq_h += 
    eq_h += f * SURFACE[U] * ds

    eq_g = (1/dt) * (g-g0) * h * dx
    eq_g += 
    eq_g += -LIHGT * CONST['REACTION'] * h * dx
    eq_g += h * SURFACE[G] * ds

    return {U: eq_h, G: eq_g}
EQUATION = create_equation()
# FIXME: clear reset KSP solver
PROBLEM = []


def set_initial():
    # TODO: reset bcs
    SUB_FUNC[U].interpolate(Function(SUBSPACE[U], 0.1))
    SUB_FUNC[G].interpolate(Function(SUBSPACE[G], 0))

    FUNC.x.scatter_forward()
    FUNC0.interpolate(FUNC)

    TIME.value = 0

    PROBLEM = NonlinearProblem(
        F=sum(EQUATION.values()),
        bcs=[],
        u=FUNC,
        solve_options={
            'convergence': 'incremental', 'tolerance': 1E-6
        },
        petsc_options={
            'ksp_type': 'preonly',
            'pc_type': 'lu',
            'pc_factor_mat_solver_type': 'mumps'
        },
        form_compiler_params={},
        jit_params={},
    )
set_initial()

def solve(n_steps, reset=True, save=False):

    def _set_next():
        # TODO: interpolate bcs
        FUNC0.interpolate(FUNC)
        TIME.value += 0.01

    def _solve_default(steps):
        for step in steps:
            _set_next()
            steps.set_description(f'Solving PDE. Time:{TIME.value:.2f}')
            PROBLEM.solve()

    if reset: set_initial()
    steps_line = tqdm(
        desc=f'Solving PDE. Time:{TIME.value:.3f}',
        iterable=np.arange(0, n_steps, dtype=int),
    )
    _solve_default(steps_line)


KeyError: 'A_NM'

In [ ]:
solve(
     # save=True,
     # n_steps=2000,
    reset=True,
)
function_plots([
    Function(SUBSPACE['N'], SUB_FUNC['N']),
    Function(SUBSPACE['N'], SUB_FUNC['N'])
])
